In [ ]:
!conda create -n aer_collab numpy pip matplotlib jupyter

In [ ]:
!source activate aer_collab

In [ ]:
!pip install -U hbp_neuromorphic_platform

In [ ]:
import nmpi
client = nmpi.Client("sesutton")
print(client.my_collabs().keys())

In [ ]:
aistate_collab_id = client.my_collabs()['UH AI state']['id']

In [ ]:
!wget http://rpg.ifi.uzh.ch/datasets/davis/office_zigzag.zip

In [ ]:
!unzip office_zigzag.zip -d office_zigzag/
!rm office_zigzag.zip
!cp office_zigzag/events.txt .
!rm -r office_zigzag/

In [ ]:
%%writefile pynnscript.py

import os
import urllib
import zipfile

directory = os.getcwd() + '/data/'
if not os.path.exists(directory):
    os.makedirs(directory)

url = 'http://rpg.ifi.uzh.ch/datasets/davis/office_zigzag.zip'
zipfn = 'data.zip'

urllib.urlretrieve(url, zipfn)
with zipfile.ZipFile( os.getcwd() + '/' + zipfn, "r") as z: 
    z.extractall(directory)

import numpy as np

fname = directory + 'events.txt'
#dt = np.dtype([('time', float), ('x',int), ('y', int) ,('polarity', int)])
event_data = np.genfromtxt(fname,delimiter=' ')

import shutil
shutil.rmtree(directory)
os.remove(zipfn)

In [ ]:
%%writefile -a pynnscript.py

#input_rect (min_x,max_x,min_y,max_y)
input_rect = (0,10,0,10)

#get data within input_rect
x_rows = np.where(np.logical_and(event_data[:,1] >= input_rect[0], event_data[:,1] <= input_rect[1]))
y_rows = np.where(np.logical_and(event_data[:,1] >= input_rect[2], event_data[:,1] <= input_rect[3]))
rows = np.intersect1d(x_rows,y_rows)
event_data = event_data[rows]

#remove on events
rows = np.where(event_data[:,3] == 0)
event_data = event_data[rows]


Based on 'https://github.com/SpiNNakerManchester/PyNN7Examples/blob/master/examples/external_devices_examples/live_examples/spike_io.py'

In [ ]:
%%writefile -a pynnscript.py

import spynnaker7.pyNN as p
import time
from threading import Condition

# initial call to set up the front end (pynn requirement)
p.setup(timestep=1.0, min_delay=1.0, max_delay=144.0)


In [ ]:
%%writefile -a pynnscript.py

# neurons per population and the length of runtime in ms for the simulation,
# as well as the expected weight each spike will contain
n_neurons = (input_rect[1] - input_rect[0]) * (input_rect[3] - input_rect[2])
n_pops = 2

input_runtime = np.max(event_data[:,0]) * 1000
extra_time = 1000
run_time = input_runtime + extra_time

weight_to_spike = 2.0


In [ ]:
%%writefile -a pynnscript.py
# neural parameters of the ifcur model used to respond to injected spikes.
# (cell params for a synfire chain)
cell_params_lif = {'cm': 0.25,
                   'i_offset': 0.0,
                   'tau_m': 20.0,
                   'tau_refrac': 2.0,
                   'tau_syn_E': 5.0,
                   'tau_syn_I': 5.0,
                   'v_reset': -70.0,
                   'v_rest': -65.0,
                   'v_thresh': -50.0
                   }

##################################
# Parameters for the injector population.  This is the minimal set of
# parameters required, which is for a set of spikes where the key is not
# important.  Note that a virtual key *will* be assigned to the population,
# and that spikes sent which do not match this virtual key will be dropped;
# however, if spikes are sent using 16-bit keys, they will automatically be
# made to match the virtual key.  The virtual key assigned can be obtained
# from the database.
##################################
cell_params_spike_injector = {
    # The port on which the spiNNaker machine should listen for packets.
    # Packets to be injected should be sent to this port on the spiNNaker
    # machine
    'port': 12345,
}


##################################
# Parameters for the injector population.  Note that each injector needs to
# be given a different port.  The virtual key is assigned here, rather than
# being allocated later.  As with the above, spikes injected need to match
# this key, and this will be done automatically with 16-bit keys.
##################################
cell_params_spike_injector_with_key = {

    # The port on which the spiNNaker machine should listen for packets.
    # Packets to be injected should be sent to this port on the spiNNaker
    # machine
    'port': 12346,

    # This is the base key to be used for the injection, which is used to
    # allow the keys to be routed around the spiNNaker machine.  This
    # assignment means that 32-bit keys must have the high-order 16-bit
    # set to 0x7; This will automatically be prepended to 16-bit keys.
    'virtual_key': 0x70000,
}


In [ ]:
%%writefile -a pynnscript.py

timing_rule = sim.SpikePairRule(tau_plus=20.0, tau_minus=20.0,
                                A_plus=0.5, A_minus=0.5)
weight_rule = sim.AdditiveWeightDependence(w_max=5.0, w_min=0.0)

stdp_model = sim.STDPMechanism(timing_dependence=timing_rule,
                               weight_dependence=weight_rule,
                               weight=0.0, delay=5.0)


In [ ]:
%%writefile -a pynnscript.py

pops = []

# Create injection populations
injector = p.Population(
        n_neurons, p.external_devices.SpikeInjector,
        cell_params_spike_injector_with_key, label='spike_injector')

for i in range(0, n_pops):
    # create populations (if cur exp)
    pops.append(p.Population(n_neurons, p.IF_curr_exp,
                                  cell_params_lif, label='pop_{}'.format(i)))

   
    # Create a connection from the injector into the populations
    p.Projection(injector, pops[i],
                        p.OneToOneConnector(weights=weight_to_spike), synapse_type=stdp_model)
    
    # record output for this population
    pop[i].record()
    
    # Activate the sending of live spikes
    p.external_devices.activate_live_output_for(
        pop[i], database_notify_host="localhost",
        database_notify_port_num=19996)
    

In [ ]:
%%writefile -a pynnscript.py
# Create a connection from the injector into the populations
p.Projection(pops[0], pops[1],
                    p.OneToOneConnector(weights=weight_to_spike), synapse_type=sim.StaticSynapse(weight=-0.75,delay=1.0))


In [ ]:
%%writefile -a pynnscript.py
# Create a condition to avoid overlapping prints
print_condition = Condition()


In [ ]:
%%writefile -a pynnscript.py
# Create an initialisation method
def init_pop(label, n_neurons, run_time_ms, machine_timestep_ms):
    print "{} has {} neurons".format(label, n_neurons)
    print "Simulation will run for {}ms at {}ms timesteps".format(
        run_time_ms, machine_timestep_ms)
    

In [ ]:
%%writefile -a pynnscript.py
# Create a sender of packets for the populations

import shed, time
s = sched.scheduler(time.time, time.sleep)

def send_input(label, sender):
    for i in range(0, event_data.shape[0]):
        neuron_id = ((event_data[i,1] - input_rect[0]) * input_rect[1]) + (event_data[i,2] - input_rect[2])
        for p in range(0, n_pops):
            neuron_id = neuron_id + (n_neurons * p)
            s.enter(event_data[i,0],0,sender.send_spike,(label,neuron_id))
    
    '''
    print_condition.acquire()
    print "Sending forward spike", neuron_id
    print_condition.release()
    sender.send_spike(label, neuron_id, send_full_keys=True)
    '''
    

In [ ]:
%%writefile -a pynnscript.py
# Create a receiver of live spikes
def receive_spikes(label, time, neuron_ids):
    for neuron_id in neuron_ids:
        print_condition.acquire()
        print "Received spike at time", time, "from", label, "-", neuron_id
        print_condition.release()
        

In [ ]:
%%writefile -a pynnscript.py
# Set up the live connection for sending spikes
live_spikes_connection_send = \
    p.external_devices.SpynnakerLiveSpikesConnection(
        receive_labels=None, local_port=19999,
        send_labels=["spike_injector"])

# Set up callbacks to occur at initialisation
live_spikes_connection_send.add_init_callback(
    "spike_injector", init_pop)

# Set up callbacks to occur at the start of simulation
live_spikes_connection_send.add_start_callback(
    "spike_injector", send_input)


In [ ]:
%%writefile -a pynnscript.py
# if not using the c visualiser, then a new spynnaker live spikes
# connection is created to define that there is a python function which
# receives the spikes.
pop_labels = ['pop_{}'.format(i) for i in range(0,n_pops)]
live_spikes_connection_receive = \
    p.external_devices.SpynnakerLiveSpikesConnection(
        receive_labels=pop_labels,
        local_port=19996, send_labels=None)

# Set up callbacks to occur when spikes are received
for i in range(0, n_pops):
    live_spikes_connection_receive.add_receive_callback(
        pop_labels[i], receive_spikes)


In [ ]:
%%writefile -a pynnscript.py
# Run the simulation on spiNNaker
p.run(run_time)

# Retrieve spikes from the synfire chain population
spikes = [pop[i].getSpikes() for i in range(0, n_pops)]

# If there are spikes, plot using matplotlib
number_of_spikes = sum([len(pop[i]) for i in range(0, n_pops)])
if number_of_spikes != 0:
    pylab.figure()
    for spike_pop in spikes:
        if len(spikes_pop) != 0:
            pylab.plot([i[1] for i in spikes_pop],
                       [i[0] for i in spikes_pop])
    pylab.ylabel('neuron id')
    pylab.xlabel('Time/ms')
    pylab.title('spikes')
    pylab.show()
else:
    print "No spikes received"

# Clear data structures on spiNNaker to leave the machine in a clean state for
# future executions
p.end()


In [ ]:
%pycat pynnscript.py

In [ ]:
job_path = client.submit_job(source='pynnscript.py',
                      platform=nmpi.SPINNAKER,
                      collab_id=aistate_collab_id,
                      config={},
                      command="run.py")


In [ ]:
job_id = job_path.split('/')[-1]

In [ ]:
# execute this line until it says 'finished'
client.job_status(job_id)

In [ ]:
ret = client.get_job(job_id, with_log=True)

In [ ]:
print(ret['log'])